### Table comparing barcoded Prom and Gridion
Generates comparison tables for unblocked and sequenced metrics for the two tables.
Goes from sequnecing summary files for these experiments, which are trimmed to only keep the columns we need, and are sorted based on the presence of the read_id in the `unblocked_read_ids.txt` from readfish.

Note for me:
fast5s for gridion run are here:
`/mnt/wastewater/scratch_backup/ML_Runs_GridION/AP_002/BC-3-Panel-Human/20211025_1338_X1_FAQ39590_af835873/fast5*`

fast5s for prom run are here:
`/mnt/waterprom/ml_runs/PAS_001/NA12878_05_NB4_06_22Rv1_07`

Sequencing summary files were created with guppy 6.4.6

Separated with `rftools` which requires the unblocked read ids file, https://github.com/LooseLab/rftools

```bash
rftools split-ss -p <PREFIX> `20220317_1702_3A_PAK09329_4cae256c` sequencing_summary_PAK09329_e742ec3b.txt
```

where <PREFIX> is one of a combination of sequencing device and run number, i.e grid1, prom2 etc.

Split sequencing summarys were trimmed to only contain relevant information using `qsvlite` - see https://github.com/jqnatividad/qsv?tab=readme-ov-file#option-3-install-with-rust
Using the select command:
```bash
fd -e.txt ed -x qsvlite select -d "\t" -o {.}_lite.tsv read_id,sample_id,sequence_length_template,barcode_arrangement {}
```

And then compressed with `xz` (in parallel with `fd`, just repalce {} with the filename to be compressed if not using `fd`

```bash
fd -tf lite -x xz -T 24 -e -z {}
```

### Alignment
Alignment was performed using `minimap2 2.26-r1175`
A script for alignment (`align_barcode_fastq`) is provided in the table 1 directory of the repository, the `map-ont` preset is used. This runs the alignment in Parallel for all files using 16 cores, so mayeb edit that.
Both PASS and FAIL data is included.
The data was called with HAC.

### For coverage on and off target
To generate files for on and off target coverage we used `mosdepth 0.3.6`.
First the reads were mapped to Hg38.p14 with all alt and unplaced contigs removed, leaving only the main chromosome assemblies. 
Inverse complements regions (for off target based coverage) to the target bed files used were generated with `bedtools v2.31.1`
This requires the bedfile to be sorted, this can be done with `bedtools sort`.

```bash
fd -e .sorted.bed -x sh -c 'bedtools complement -i -g /data/refs/hg38_simple.genome > {.}_inverse_complement.bed'
```
These are located in the target_beds.

Mosdepth was used to calculate median coverage over the regions, and the median of this coverage was taken in this notebook. The targets were correct for each barcode. And example command:
```bash
mosdepth -m -t 32 -b CancerPanelTargets.sorted.bed -xn all_barcode_07_sequenced ../merged_reads/all_barcode07.sequenced.fastq.bam
```

An example script to run this in parallel for the Gridion run data is provided, `mosdepth_bam_files`. Again in parallel so user beware thread counts.

##### Unclassified coverage
For unclassified on/off target coverage, all three gene panel target bed files were merged, and overlaps were collapsed, but with strand awareness for the merging.
Merging was performed with `bedtools v2.31.1` using the following command - 
```bash
fd -ae sorted.bed -X cat {} | save -f all_targets.bed
bedtools sort -i all_targets.bed | save -f all_targets.sorted.bed
bedtools merge -s -i all_targets.sorted.bed | save -f all_targets.merged.sorted.bed
bedtools complement -i all_targets.merged.sorted.bed -g /data/refs/hg38_simple.genome | save -f all_targets.sorted_inverse_complement.bed
```

The script for mosdepth also does the coverage for on/off target coverage in unclassified.

## Warning
This notebook is memory hungry (>20Gb RAM required). If this is an issue, contact us and we will work out a way to calculate these metrics without loading everything into a dataframe.

In [213]:
import gc
import re
import sys
from itertools import chain, repeat, zip_longest
from pathlib import Path

import numpy as np
import pandas as pd

# Define some constant values which are used in the table for information
PAT = re.compile(r"(sequenced|unblocked)")
gm, nb, rv = (
    ["GM12878", "TruSight 170 Tumour Panel", 170],
    ["NB4", "TruSight RNA Fusion Panel", 508],
    ["22Rv1", "COSMIC", 717],
)
gridion_samples = {
    "barcode01": gm,
    "barcode02": nb,
    "barcode03": rv,
    "unclassified_grid": ["*", "*", "*"],
}
prom_samples = {
    "barcode05": gm,
    "barcode06": nb,
    "barcode07": rv,
    "unclassified_prom": ["*", "*", "*"],
}
# Combine which will be used in final table
prom_samples.update(gridion_samples)

In [2]:
def n50(lengths: pd.Series) -> float:
    """
    Returns the N50 of a series of read lengths
    """
    lengths = lengths.sort_values(ascending=False).values
    cumulative_sum = lengths.cumsum()
    total_length = cumulative_sum[-1]
    n50_length = lengths[(cumulative_sum >= total_length / 2).argmax()]
    return n50_length

#### Read in all the xz compressed subset sequencing summary files for GridION experiment

In [3]:
dfs = []
for summary in Path(".").rglob("gridion*.tsv.xz"):
    read_type = PAT.findall(summary.name)[0]
    df = pd.read_csv(summary, sep="\t")
    df["read_type"] = read_type
    dfs.append(df)

gridion_df = pd.concat(dfs)

In [212]:
# Calculate metrics for Gridion read lengths, yield and N50
gridion_summary = gridion_df.groupby(["read_type", "barcode_arrangement"])[
    "sequence_length_template"
].agg([np.mean, np.sum, n50])
# Only select relevant barcodes
gridion_summary = gridion_summary.loc[
    :, ["barcode01", "barcode02", "barcode03", "unclassified"], :
]

/tmp/ipykernel_1452772/3826483749.py:4: FutureWarning: The provided callable <function mean at 0x7f7f344cff40> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ].agg([np.mean, np.sum, n50])
/tmp/ipykernel_1452772/3826483749.py:4: FutureWarning: The provided callable <function sum at 0x7f7f344ceb00> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ].agg([np.mean, np.sum, n50])


## Promethion
#### Read in all the xz compressed subset sequencing summary files for PromethION experiment

In [6]:
dfs = []
for summary in Path(".").rglob("sequencing_summaries/prom*.tsv.xz"):
    read_type = PAT.findall(summary.name)[0]
    df = pd.read_csv(summary, sep="\t")
    df["read_type"] = read_type
    dfs.append(df)

prom_df = pd.concat(dfs)

In [7]:
prom_summary = prom_df.groupby(["read_type", "barcode_arrangement"])[
    "sequence_length_template"
].agg([np.mean, np.sum, n50])
# Select correct barcodes
prom_summary = prom_summary.loc[
    :, ["barcode05", "barcode06", "barcode07", "unclassified"], :
]

/tmp/ipykernel_1452772/4195199970.py:3: FutureWarning: The provided callable <function mean at 0x7f7f344cff40> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ].agg([np.mean, np.sum, n50])
/tmp/ipykernel_1452772/4195199970.py:3: FutureWarning: The provided callable <function sum at 0x7f7f344ceb00> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ].agg([np.mean, np.sum, n50])


In [9]:
gridion_summary = gridion_summary.reset_index()
gridion_summary["barcode_arrangement"] = gridion_summary["barcode_arrangement"].map(
    lambda x: x if x != "unclassified" else "unclassified_grid"
)
gridion_summary["Device"] = "G."

In [10]:
prom_summary = prom_summary.reset_index()
prom_summary["barcode_arrangement"] = prom_summary["barcode_arrangement"].map(
    lambda x: x if x != "unclassified" else "unclassified_prom"
)
prom_summary["Device"] = "P."

In [11]:
testing = pd.concat((gridion_summary, prom_summary))

In [12]:
testing = testing.rename(
    columns={
        "barcode_arrangement": "Barcode",
        "mean": "Mean",
        "sum": "Yield",
        "n50": "N50",
    }
)
testing["read_type"] = testing["read_type"].map(
    {"sequenced": "Seq.", "unblocked": "Unb."}
)

In [219]:
# Pivot from long form to wideform, keeping only relevant columns, more to be added later
t = testing.pivot(
    index=["Barcode", "Device"],
    columns="read_type",
    values=[
        "Yield",
        "Mean",
        "N50",
    ],
)
# Assign the panel information
t[["Sample", "Panel", "Gene number"]] = pd.DataFrame.from_records(
    t.index.get_level_values(0).map(prom_samples), index=t.index
)

#### Get the regions from the mosdepth files


In [220]:
def read_coverages_of_regions() -> pd.DataFrame:
    """Read the mosdepth coverage region bed files filtering to the given barcodes.
    Create a concatenated datafrom of all files, with barcode, coverage and on and off target and return it.
    """
    prom = ("prom", "05", "06", "07", "unclassified_prom")
    grid = ("grid", "01", "02", "03", "unclassified_grid")
    pat = re.compile(r"(barcode|unclassified[_promgrid]{5})_*(\d{2})*.*")
    coverages = []
    for barcode, device in chain(
        zip_longest(prom[1:], [], fillvalue=prom[0]),
        zip_longest(grid[1:], [], fillvalue=grid[0]),
    ):
        files = list(Path("mosdepth_region_summaries/").glob(f"*{barcode}*.bed.gz"))
        # print(files)
        for region_file in files:
            region_type = (
                "Off target"
                if "off_target" in str(region_file) or "inverse" in str(region_file)
                else "On target"
            )
            print(f"\t{region_file}\t{region_type}")
            barcode = "".join(pat.findall(str(region_file))[0])
            # Unclassified and off target bed files do not provide names for records so we shift index for Coverage column down one
            usecols = (
                [3]
                if region_type == "Off target" or "unclassified" in str(region_file)
                else [4]
            )
            df = pd.read_csv(
                region_file, sep="\t", header=None, usecols=usecols, names=["Coverage"]
            )
            df["barcode"] = barcode
            df["region_type"] = region_type
            #             P. or G. to match the barcode/device assignement in main dataframe
            df["Device"] = f"{device[0].upper()}."
            coverages.append(df)
    return pd.concat(coverages)

In [221]:
coverage_df = read_coverages_of_regions()

	mosdepth_region_summaries/all_barcode_05.regions.bed.gz	On target
	mosdepth_region_summaries/all_barcode_05_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/all_barcode_06.regions.bed.gz	On target
	mosdepth_region_summaries/all_barcode_06_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/all_barcode_07.regions.bed.gz	On target
	mosdepth_region_summaries/all_barcode_07_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/unclassified_prom_off_target.regions.bed.gz	Off target
	mosdepth_region_summaries/unclassified_prom_on.regions.bed.gz	On target
	mosdepth_region_summaries/barcode01_tst_170_coords_extended.sorted.regions.bed.gz	On target
	mosdepth_region_summaries/barcode01_tst_170_coords_extended.sorted_inverse_complement.regions.bed.gz	Off target
	mosdepth_region_summaries/barcode02_tst_fusion_coords_extended.sorted_inverse_complement.regions.bed.gz	Off target
	mosdepth_region_summaries/barcode02_tst_fusion_coords_extended.sorted.regions.bed.gz

#### Caluculate median coverage from the mosdepth files and get it into a format which can be ampped to amin dataframe via the index

In [222]:
a = (
    coverage_df.groupby(["barcode", "region_type", "Device"])["Coverage"]
    .median()
    .reset_index()
    .pivot(index=["barcode", "Device"], columns="region_type", values="Coverage")
)
a.index.names = ["Barcode", "Device"]

In [223]:
# Assign coverages
t[["Off Target Median", "Target Median"]] = a
# Reorder the rows to this order
t = t.reindex(
    [
        ("barcode01", "G."),
        ("barcode02", "G."),
        ("barcode03", "G."),
        ("unclassified_grid", "G."),
        ("barcode05", "P."),
        ("barcode06", "P."),
        ("barcode07", "P."),
        ("unclassified_prom", "P."),
    ]
)
# Assign absolute fold difference bewteen on and off target coverage
t["Fold Enrichment"] = t["Target Median"] / t["Off Target Median"]
# Not used for the unclassified fields
t.loc[["unclassified_grid", "unclassified_prom"], "Fold Enrichment"] = "*"

/tmp/ipykernel_1452772/2991135328.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '*' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  t.loc[["unclassified_grid", "unclassified_prom"], "Fold Enrichment"] = "*"


#### Calculate the values for the total rows of the final table and assign them

In [224]:
total_df = (
    t.groupby(level=1)
    .agg(
        {
            ("Mean", "Seq."): "mean",
            ("Mean", "Unb."): "mean",
            ("Yield", "Seq."): "sum",
            ("Yield", "Unb."): "sum",
            ("N50", "Seq."): "mean",
            ("N50", "Unb."): "mean",
            ("Off Target Median", ""): "median",
            ("Target Median", ""): "median",
        }
    )
    .reset_index()
)
total_df["Barcode"] = "Total"
t = pd.concat((t, total_df.set_index(["Barcode", "Device"])))
#  Reorder one more time
t = t.reindex(
    [
        ("barcode01", "G."),
        ("barcode02", "G."),
        ("barcode03", "G."),
        ("unclassified_grid", "G."),
        ("Total", "G."),
        ("barcode05", "P."),
        ("barcode06", "P."),
        ("barcode07", "P."),
        ("unclassified_prom", "P."),
        ("Total", "P."),
    ]
)

In [225]:
t.head(12)

Yield                       Mean              \
read_type                         Seq.          Unb.         Seq.        Unb.   
Barcode           Device                                                        
barcode01         G.      3.345244e+08  3.465450e+09  1926.417947  554.378839   
barcode02         G.      1.238984e+09  4.839475e+09  4203.349585  550.872480   
barcode03         G.      1.248408e+09  3.835499e+09  5065.192309  555.972116   
unclassified_grid G.      1.709108e+08  3.658782e+09   798.171085  733.417091   
Total             G.      2.992827e+09  1.579921e+10  2998.282731  598.660132   
barcode05         P.      1.278234e+09  1.476375e+10   930.660416  897.375697   
barcode06         P.      4.357606e+09  2.339281e+10  1914.922736  903.235613   
barcode07         P.      3.010605e+09  1.363278e+10  2224.730111  909.816885   
unclassified_prom P.      7.036909e+08  1.549620e+10   507.391720  962.700204   
Total             P.      9.350136e+09  6.728553e+10  1394.426246  918.282100   

                              N50           Sample                      Panel  \
read_type                    Seq.    Unb.                                       
Barcode           Device                                                        
barcode01         G.      8149.00  555.00  GM12878  TruSight 170 Tumour Panel   
barcode02         G.      7191.00  552.00      NB4  TruSight RNA Fusion Panel   
barcode03         G.      6858.00  556.00    22Rv1                     COSMIC   
unclassified_grid G.       923.00  792.00        *                          *   
Total             G.      5780.25  613.75      NaN                        NaN   
barcode05         P.      7163.00  917.00  GM12878  TruSight 170 Tumour Panel   
barcode06         P.      7349.00  919.00      NB4  TruSight RNA Fusion Panel   
barcode07         P.      6999.00  923.00    22Rv1                     COSMIC   
unclassified_prom P.       543.00  989.00        *                          *   
Total             P.      5513.50  937.00      NaN                        NaN   

                         Gene number Off Target Median Target Median  \
read_type                                                              
Barcode           Device                                               
barcode01         G.             170             1.000        12.000   
barcode02         G.             508             1.000        15.000   
barcode03         G.             717             1.000        12.000   
unclassified_grid G.               *             0.000         1.000   
Total             G.             NaN             1.000        12.000   
barcode05         P.             170             4.000        35.000   
barcode06         P.             508             7.635        52.000   
barcode07         P.             717             4.000        27.250   
unclassified_prom P.               *             4.000         5.000   
Total             P.             NaN             4.000        31.125   

                         Fold Enrichment  
read_type                                 
Barcode           Device                  
barcode01         G.                12.0  
barcode02         G.                15.0  
barcode03         G.                12.0  
unclassified_grid G.                   *  
Total             G.                 NaN  
barcode05         P.                8.75  
barcode06         P.             6.81074  
barcode07         P.              6.8125  
unclassified_prom P.                   *  
Total             P.                 NaN

#### Define some functions to format the cells for the latex table
Otherwise cells are impossible to read

In [208]:
def format_bases(num: int, factor: int = 1000, suffix: str = "B") -> str:
    """Return a human readable string of a large number using SI unit prefixes

    :pararm num: A number to convert to decimal form
    :param factor: The SI factor, use 1000 for SI units and 1024 for binary multiples
    :param suffix: The suffix to place after the SI prefix, for example use B for SI units and iB for binary multiples
    :return: The input number formatted to two decimal places with the SI unit and suffix

    :Example:

    >>> format_bases(1_000)
    '1.00 kB'
    >>> format_bases(1_000_000)
    '1.00 MB'
    >>> format_bases(1_630_000)
    '1.63 MB'
    >>> format_bases(1_000_000_000)
    '1.00 GB'
    """
    if isinstance(num, float) or isinstance(num, int):
        for unit in ["", "k", "M", "G", "T", "P", "E", "Z"]:
            if abs(num) < factor:
                return f"{num:3.2f}{unit}{suffix}"
            num /= factor
        return f"{num:3.2f}"
    else:
        return num


def format_fold(x: int) -> str:
    """
    Format the fold change
    """
    if not isinstance(x, float):
        return x
    # Check if the value is an integer
    if isinstance(x, (int, float)) and x == int(x):
        return f"{int(x)}x"
    # Otherwise, format to one decimal place
    elif isinstance(x, float):
        return f"{x:.1f}x"
    return x

#### Creat the latex table, and set up all the environments to make it look pretty.
NB. I had to manually tweak some formatting the manuscript, such as making the MegaBase yields into 0.XXX (GB), and making the Total rows bold, and removing the End Reason cell and changing it to the word barcode

In [210]:
t = t.fillna("")
t.columns.names = [None, "End Reason"]
idx = pd.IndexSlice
subset = idx[
    :,
    [(metric, end_reason) for end_reason in ("Seq.", "Unb.") for metric in ["Yield"]],
]
idx2 = pd.IndexSlice
subset2 = idx2[:, ("Fold Enrichment", "")]
idx3 = pd.IndexSlice
subset3 = idx3[
    :,
    [
        ("Yield", "Seq."),
        ("Yield", "Unb."),
        ("N50", "Seq."),
        ("N50", "Unb."),
        ("Off Target Median", ""),
        ("Target Median", ""),
        ("Fold Enrichment", ""),
    ],
]
s = (
    (
        # Not proud of this loc, in here to exlude the mean columns, because I couldn't manage the slice to get the to hide
        t.loc[:, ~t.columns.isin([("Mean", "Seq."), ("Mean", "Unb.")])]
        .style.hide(axis=0, level=1, names=True)
#          Format all numerical values to no decimal points
        .format(precision=0, subset=subset3, thousands=",")
        .format(format_bases, subset=subset)
        # .format(
        #     "\\textbf{{{}}}",
        #     subset=idx[
        #         [("Total", "G."), ("Total", "P.")],
        #         :,
        #     ],
        #     precision=0,
        #     thousands=","
        # )
        .relabel_index(
            t.loc[:, ~t.columns.isin([("Mean", "Seq."), ("Mean", "Unb.")])]
            .columns.get_level_values(0)
            .map(lambda x: "Yield\n(Gb)" if str(x) == "Yield" else x),
            axis=1,
            level=0,
        )
        .format_index(escape="latex", axis=0)
#         Make columns label bold
        .format_index("\\textbf{{{}}}", escape="latex", axis=1, level=[0, 1])
        .to_latex(
            position_float="centering",
            hrules=True,
            column_format="".join(repeat("X ", len(t.columns) - 2 + 1)),
            environment="table",
        )
    )
#     Manual replaces of the latex string
    .replace(r"\begin{tabular}", r"\begin{tabularx}{\textwidth}")
    .replace("\centering", "\centering\n\scriptsize")
    .replace("unclassified\_grid", "unclassified")
    .replace("unclassified\_prom", "unclassified")
    .replace("Total & 2.99", "\midrule\nTotal & 2.99")
    .replace("Total", r"\textbf{Total}")
)

In [211]:
print(s)

\begin{table}
\centering
\scriptsize
\begin{tabularx}{\textwidth}{X X X X X X X X X X X}
\toprule
 & \multicolumn{2}{r}{\textbf{Yield}} & \multicolumn{2}{r}{\textbf{N50}} & \textbf{Sample} & \textbf{Panel} & \textbf{Gene number} & \textbf{Off Target Median} & \textbf{Target Median} & \textbf{Fold Enrichment} \\
End Reason & \textbf{Seq.} & \textbf{Unb.} & \textbf{Seq.} & \textbf{Unb.} & \textbf{} & \textbf{} & \textbf{} & \textbf{} & \textbf{} & \textbf{} \\
\midrule
barcode01 & 334.52MB & 3.47GB & 8,149 & 555 & GM12878 & TruSight 170 Tumour Panel & 170 & 1 & 12 & 12 \\
barcode02 & 1.24GB & 4.84GB & 7,191 & 552 & NB4 & TruSight RNA Fusion Panel & 508 & 1 & 15 & 15 \\
barcode03 & 1.25GB & 3.84GB & 6,858 & 556 & 22Rv1 & COSMIC & 717 & 1 & 12 & 12 \\
unclassified & 170.91MB & 3.66GB & 923 & 792 & * & * & * & 0 & 1 & * \\
\midrule
\textbf{Total} & 2.99GB & 15.80GB & 5,780 & 614 &  &  &  & 1 & 12 &  \\
barcode05 & 1.28GB & 14.76GB & 7,163 & 917 & GM12878 & TruSight 170 Tumour Panel & 170 & 